## Imports

In [1]:
import gdown
import os
import pathlib
import re
import shutil
import sys

from sklearn.model_selection import train_test_split
from tqdm import tqdm

#### Add parent directory to path to enable access to other scripts)

In [2]:
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Download Rellis-3D Data
Change dataset_path if you want to download to a different directory

In [3]:
dataset_path = os.path.join(os.getcwd(), "..", "datasets", "Rellis")
print(dataset_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis


## Download Images

In [4]:
rellis_images_path = os.path.join(dataset_path, "Rellis_3D_pylon_camera_node.zip")
print(rellis_images_path)
gdown.download("https://drive.google.com/uc?id=1F3Leu0H_m6aPVpZITragfreO_SGtL2yV", rellis_images_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\Rellis_3D_pylon_camera_node.zip


## Download Labels

In [5]:
rellis_labels_path = os.path.join(dataset_path, "Rellis_3D_pylon_camera_node_label_color.zip")
print(rellis_labels_path)
gdown.download("https://drive.google.com/uc?id=1HJl8Fi5nAjOr41DPUFmkeKWtDXhCZDke", rellis_labels_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\Rellis_3D_pylon_camera_node_label_color.zip


## Unzip images and labels

In [6]:
rellis_extract_path = os.path.join(dataset_path, "extracted")
print(rellis_extract_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\extracted


In [7]:
rellis_images_extract_path = os.path.join(rellis_extract_path, "images")
print(rellis_images_extract_path)
gdown.extractall(rellis_images_path, rellis_images_extract_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\extracted\images


In [8]:
rellis_labels_extract_path = os.path.join(rellis_extract_path, "labels")
print(rellis_labels_extract_path)
gdown.extractall(rellis_labels_path, rellis_labels_extract_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\extracted\labels


# Clean Up File Structure

### Helper Methods

In [9]:
def split_name(filename):
    parts = re.split(r"\.|/|\\", str(filename))
    return (parts[-2], parts[-1])

def consolidate_files(dataset_dir, new_dir, file_ext):
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
        files = list(pathlib.Path(dataset_dir).glob(f"**/*.{file_ext}"))
        print(f"{len(files)} files with extension .{file_ext} found")
        for f in tqdm(files):
            file_name = ".".join(split_name(f))
            shutil.move(f, os.path.join(new_dir, file_name))
    else:
        print(f"Directory already exists at path: {new_dir}")

def remove_unpaired_images(image_dir, label_dir):
    if not os.path.exists(image_dir) or not os.path.exists(label_dir):
        return
    
    image_set = set(split_name(image)[0] for image in pathlib.Path(image_dir).glob("*"))
    label_set = set(split_name(label)[0] for label in pathlib.Path(label_dir).glob("*"))
    unpaired_images = image_set.difference(label_set)
    print(f"Deleting {len(unpaired_images)} unpaired images")

    for im in unpaired_images:
        image_path = os.path.join(image_dir, f"{im}.jpg")
        if os.path.exists(image_path):
            os.remove(image_path)

In [10]:
rellis_processed_images_path = os.path.join(dataset_path, "images")
print(rellis_processed_images_path)
rellis_processed_labels_path = os.path.join(dataset_path, "labels")
print(rellis_processed_labels_path)

consolidate_files(rellis_extract_path, rellis_processed_images_path, "jpg")
consolidate_files(rellis_extract_path, rellis_processed_labels_path, "png")

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\images
d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels


In [11]:
remove_unpaired_images(rellis_processed_images_path, rellis_processed_labels_path)

### For splitting data

In [12]:
def get_data_splits(image_dir, test_size=0.10, val_size=0.05, random_state=123):
    files = [split_name(filename)[0] for filename in pathlib.Path(image_dir).glob("*")]
    actual_test_size = val_size + test_size
    actual_val_size  = val_size / actual_test_size
    train, test = train_test_split(files, test_size=actual_test_size, random_state=random_state)
    test, val   = train_test_split(test,  test_size=actual_val_size,  random_state=random_state)
    print(len(train), len(test), len(val))
    return train, test, val

def make_data_dirs(image_dir, label_dir):
    for dir_name in [image_dir, label_dir]:
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        for split_name in ["train", "test", "val"]:
            split_dir = os.path.join(dir_name, split_name)
            if not os.path.exists(split_dir):
                os.mkdir(split_dir)

def move_files(train, test, val, image_dir, label_dir):
    make_data_dirs(image_dir, label_dir)

    for dir_name, data_split in [("train", train), ("test", test), ("val", val)]:
        for filename in tqdm(data_split, desc=dir_name):
            image_name = f"{filename}.jpg"
            label_name = f"{filename}.png"

            shutil.move(
                os.path.join(image_dir, image_name),
                os.path.join(image_dir, dir_name, image_name)
            )
            shutil.move(
                os.path.join(label_dir, label_name),
                os.path.join(label_dir, dir_name, label_name)
            )

In [13]:
train, test, val = get_data_splits(rellis_processed_images_path)

In [14]:
move_files(train, test, val, rellis_processed_images_path, rellis_processed_labels_path)

# Run scripts

## Generate Instance Ids

In [15]:
from createPanopticInstanceIds import main as cpii_main
cpii_main([rellis_processed_labels_path, "rellis", 12])

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 6234/6234 [21:17<00:00,  4.88it/s] 

6234 files generated
TOOK 1277.642659664154 SECONDS!


## Generate Panoptic Annotations

In [18]:
from createPanopticAnnotations import main as cpa_main

processed_labels_train_path = os.path.join(rellis_processed_labels_path, "train")
print(processed_labels_train_path)
#cpa_main([processed_labels_train_path, "rellis"])

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\train


Generating Panoptic Images:   0%|          | 1/5298 [00:00<21:53,  4.03it/s]


KeyError: 255

In [ ]:
processed_labels_test_path = os.path.join(rellis_processed_labels_path, "test")
print(processed_labels_test_path)
#cpa_main([processed_labels_test_path, "rellis"])


Saving the json file Rellis_labels\annotations_test_panoptic.json
TOOK 119.45091462135315 SECONDS!



Generating Panoptic Images: 100%|██████████| 624/624 [01:59<00:00,  5.23it/s]


In [ ]:
processed_labels_val_path = os.path.join(rellis_processed_labels_path, "val")
print(processed_labels_val_path)
#cpa_main([processed_labels_val_path, "rellis"])


Saving the json file Rellis_labels\annotations_val_panoptic.json
TOOK 58.584067821502686 SECONDS!



Generating Panoptic Images: 100%|██████████| 311/311 [00:58<00:00,  5.31it/s]


## Generate Instance Annotations

In [ ]:
#!python Offroad-Panoptic-Segmentation/createInstances.py Rellis_labels/train/


Generating Panoptic Images:  22%|██▏       | 1143/5298 [01:50<06:42, 10.33it/s]
Traceback (most recent call last):
  File "d:\Github Repos\SJSU\CMPE295\RELLIS\Offroad-Panoptic-Segmentation\createInstances.py", line 155, in <module>
    generatePanopticImages(input_dir)
  File "d:\Github Repos\SJSU\CMPE295\RELLIS\Offroad-Panoptic-Segmentation\createInstances.py", line 92, in generatePanopticImages
    poly = Polygon(contour)
  File "d:\Github Repos\SJSU\CMPE295\.venv\lib\site-packages\shapely\geometry\polygon.py", line 261, in __init__
    ret = geos_polygon_from_py(shell, holes)
  File "d:\Github Repos\SJSU\CMPE295\.venv\lib\site-packages\shapely\geometry\polygon.py", line 539, in geos_polygon_from_py
    ret = geos_linearring_from_py(shell)
  File "shapely\speedups\_speedups.pyx", line 257, in shapely.speedups._speedups.geos_linearring_from_py
ValueError: A LinearRing must have at least 3 coordinate tuples


In [ ]:
#!python Offroad-Panoptic-Segmentation/createInstances.py Rellis_labels/test/


Saving the json file Rellis_labels\annotations_test_instances.json
TOOK 56.159759283065796 SECONDS!



Generating Panoptic Images: 100%|██████████| 624/624 [00:56<00:00, 11.13it/s]


In [ ]:
#!python Offroad-Panoptic-Segmentation/createInstances.py Rellis_labels/val/


Saving the json file Rellis_labels\annotations_val_instances.json
TOOK 26.644744396209717 SECONDS!



Generating Panoptic Images: 100%|██████████| 311/311 [00:26<00:00, 11.71it/s]


## Generate Semantic Segmentation Images

In [ ]:
#!git clone https://github.com/cocodataset/panopticapi.git

Cloning into 'panopticapi'...


In [ ]:
#%pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to c:\users\nated\appdata\local\temp\pip-req-build-dyin71xx
  Resolved https://github.com/cocodataset/panopticapi.git to commit 7bb4655548f98f3fedc07bf37e9040a992b054b0
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for panopticapi, since package 'wheel' is not installed.
  Running setup.py install for panopticapi: started
  Running setup.py install for panopticapi: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/panopticapi.git 'C:\Users\nated\AppData\Local\Temp\pip-req-build-dyin71xx'


In [ ]:
#!python panopticapi/converters/panoptic2semantic_segmentation.py \
#--input_json_file Rellis_labels/annotations_val_panoptic.json \
#--segmentations_folder Rellis_labels/val_panoptic/ \
#--semantic_seg_folder Rellis_labels/val_semantic \
#--categories_json_file Rellis_labels/categories.json

EXTRACTING FROM...
COCO panoptic format:
	Segmentation folder: Rellis_labels/val_panoptic/
	JSON file: Rellis_labels/annotations_val_panoptic.json
SEMANTIC SEGMENTATION
in PNG format:
	Folder with semnatic segmentations: Rellis_labels/val_semantic
Creating folder Rellis_labels/val_semantic for semantic segmentation PNGs


Number of cores: 20, images per core: 16
Core: 0, 0 from 16 images processed
Core: 1, 0 from 16 images processed
Core: 2, 0 from 16 images processed
Core: 3, 0 from 16 images processed
Core: 4, 0 from 16 images processed
Core: 5, 0 from 16 images processed
Core: 6, 0 from 16 images processed
Core: 7, 0 from 16 images processed
Core: 8, 0 from 16 images processed
Core: 9, 0 from 16 images processed
Core: 10, 0 from 16 images processed
Core: 11, 0 from 15 images processed
Core: 12, 0 from 15 images processed
Core: 13, 0 from 15 images processed
Core: 14, 0 from 15 images processed
Core: 15, 0 from 15 images processed
Core: 16, 0 from 15 images processed
Core: 17, 0 from

In [ ]:
#!python panopticapi/converters/panoptic2semantic_segmentation.py \
#--input_json_file Rellis_labels/annotations_test_panoptic.json \
#--segmentations_folder Rellis_labels/test_panoptic/ \
#--semantic_seg_folder Rellis_labels/test_semantic \
#--categories_json_file Rellis_labels/categories.json

EXTRACTING FROM...
COCO panoptic format:
	Segmentation folder: Rellis_labels/test_panoptic/
	JSON file: Rellis_labels/annotations_test_panoptic.json
SEMANTIC SEGMENTATION
in PNG format:
	Folder with semnatic segmentations: Rellis_labels/test_semantic
Creating folder Rellis_labels/test_semantic for semantic segmentation PNGs


Number of cores: 20, images per core: 32
Core: 0, 0 from 32 images processed
Core: 1, 0 from 32 images processed
Core: 2, 0 from 32 images processed
Core: 3, 0 from 32 images processed
Core: 4, 0 from 31 images processed
Core: 5, 0 from 31 images processed
Core: 6, 0 from 31 images processed
Core: 7, 0 from 31 images processed
Core: 8, 0 from 31 images processed
Core: 9, 0 from 31 images processed
Core: 10, 0 from 31 images processed
Core: 11, 0 from 31 images processed
Core: 12, 0 from 31 images processed
Core: 13, 0 from 31 images processed
Core: 14, 0 from 31 images processed
Core: 15, 0 from 31 images processed
Core: 16, 0 from 31 images processed
Core: 17, 0 

In [ ]:
#!python panopticapi/converters/panoptic2semantic_segmentation.py \
#--input_json_file Rellis_labels/annotations_train_panoptic.json \
#--segmentations_folder Rellis_labels/train_panoptic/ \
#--semantic_seg_folder Rellis_labels/train_semantic \
#--categories_json_file Rellis_labels/categories.json

EXTRACTING FROM...
COCO panoptic format:
	Segmentation folder: Rellis_labels/train_panoptic/
	JSON file: Rellis_labels/annotations_train_panoptic.json
SEMANTIC SEGMENTATION
in PNG format:
	Folder with semnatic segmentations: Rellis_labels/train_semantic
Creating folder Rellis_labels/train_semantic for semantic segmentation PNGs


Number of cores: 20, images per core: 265
Core: 0, 0 from 265 images processed
Core: 1, 0 from 265 images processed
Core: 2, 0 from 265 images processed
Core: 3, 0 from 265 images processed
Core: 4, 0 from 265 images processed
Core: 5, 0 from 265 images processed
Core: 6, 0 from 265 images processed
Core: 7, 0 from 265 images processed
Core: 8, 0 from 265 images processed
Core: 9, 0 from 265 images processed
Core: 10, 0 from 265 images processed
Core: 11, 0 from 265 images processed
Core: 12, 0 from 265 images processed
Core: 13, 0 from 265 images processed
Core: 14, 0 from 265 images processed
Core: 15, 0 from 265 images processed
Core: 16, 0 from 265 images 